# BMEN 509 Bone Project

This document contains the image processing completed on the micro-CT bone core samples acquirred in the Manske lab.

Outlined in this notebook is the image processing pipeline that takes in NIFTI files converted to density units and

#### -Registers micro CT pre and post loaded bone samples 

#### -Segments pre and post loaded samples to a user defined bone threshold

#### -Overlays the pre and post samples for microfracture comparison

#### -Calculates % overlay

The notebook does this for one set of samples but was adapted for other samples

Created by Parker Nesdoly, Lauren Brown, Amelia Woodard

Last Edit - April 10, 2019

## Importing Libraries and Setting Up Constants

In [ ]:
# Installing and importing necessary libraries used within processing pipeline

!pip install pydicom
!pip install nibabel
!pip install opencv-python
!pip install SimpleITK

import os
import numpy as np
import matplotlib.pyplot as plt
import nibabel as nib
import pydicom
import cv2
import SimpleITK as sitk
import matplotlib.patches as mpatches

In [ ]:
# Defining files name and data directory variables
data_directory = os.path.join('.', 'nii')
file_names = [
    'nPost5.2C0016420.nii',
    'nPost5.3C0016421.nii',  
    'nPre5.2C0016343.nii', 'nPre5.3C0016346.nii', 
]

In [ ]:
# Define necessary processing constants to be consistent throughout the notebook

threshold=550 # density threshold value

plot_slice = 300 #This is the # slice we are looking at

## Defining Functions

In [ ]:
#Check all files are there and named properly
for file_name in file_names:
    name = os.path.join(data_directory, file_name)
    if not os.path.isfile(name):
        os.sys.exit('Cannot find file {}. Please make sure you have downloaded the data'.format(name))
print('Found all image files!')

In [ ]:
def segment(image, threshold):
    "Segmentation function used to compute a threshold segmentation mask and return image mask"
    return image>=threshold # return threshold mask

In [ ]:
def plotAll(image, slice_number):
    "Function to plot all views of an image at a defined slice add cmap = grey if desired"
    
    plt.subplots(1, 3, figsize=(15, 15))
    plt.subplot(1, 3, 1); plt.imshow(sitk.GetArrayFromImage(image[:, :, slice_number]), cmap = "gray"); plt.title("View 1", fontsize = 16)
    plt.subplot(1, 3, 2); plt.imshow(sitk.GetArrayFromImage(image[:, slice_number, :]), cmap = "gray"); plt.title("View 2", fontsize = 16)
    plt.subplot(1, 3, 3); plt.imshow(sitk.GetArrayFromImage(image[slice_number, :, :]), cmap = "gray"); plt.title("View 3", fontsize = 16)
    

In [ ]:
def resample (fixed, moving, trfm):
    """Function to resample a moving image through a determined trasnform in relation to a fixed image"""
    resampler = sitk.ResampleImageFilter()
    resampler.SetReferenceImage(fixed) # set the fixed image to be the fixed image
    resampler.SetInterpolator(sitk.sitkBSpline) # interpolating using BSpline
    resampler.SetDefaultPixelValue(0) # Anything outside is set to zero 
    resampler.SetTransform(trfm) #Set transform function as trfm

    outImg = resampler.Execute(moving) #we are executing the filter on the input image
    return (outImg) #return the output image

In [ ]:
def command_iteration(method) :
    """Function to use as additional comming registration method to print out registration values"""
    print("{0:3} = {1:10.5f} : {2}".format(method.GetOptimizerIteration(),
                                           method.GetMetricValue(),
                                           method.GetOptimizerPosition()), flush=True)

In [ ]:
def overlay(image1, image2):
    "Function that takes two segmented images and overlays them to see corresponding areas"
    
    image1 = image1*2 # Converting image 1 to 2's and zeros
    overlayImage = image1 + image2
 
    #overlayImage = overlayImage == 1 # use for only the residual 1's
    #overlayImage = overlayImage == 2 # use for only the residual 2's
    #overlayImage = overlayImage == 3 # use for only the the overlapping/alligned areas
    #overlayImage = overlayImage == 1 or 2 # use for Only the non-overlapping areas
    
    # Use this section to see the non overlapping areas in different colours
    #overlayImage1 = overlayImage == 1
    #overlayImage2 = (overlayImage == 2)*2
    #overlayImage = overlayImage1+overlayImage2
    
    return overlayImage
    

## Reading in Images and Implementing Manual Initial Transform Iteration

In [ ]:
# Reading in fixed and moving CT images (NIFTI) through SITK
post52 = sitk.ReadImage(os.path.join(data_directory, 'nPost5.2C0016420.nii'))
pre52 = sitk.ReadImage(os.path.join(data_directory, 'nPre5.2C0016343.nii'))

fixed = post52
moving = pre52

In [ ]:
tx = sitk.Transform(fixed.GetDimension(), sitk.sitkEuler)#this is our transform, it will return the dimensions of the transform, using the euler transform

# Initial transform values found through visual estimation and trial and error
tx.SetParameters( [np.pi, 0.0, 0, 1.5, 0.0, 0.0] ) # we have 6 degrees of freedom, 3 translational and 3 rotational through xyz

# Changing transform point around center rather than an arbitrary location
center_cont = [float(x)/2.0 for x in moving.GetSize()]
center = list(moving.TransformContinuousIndexToPhysicalPoint(center_cont)) + [0]
tx.SetFixedParameters(center)

# Second initial transform manual setting
tx2 = sitk.Transform(fixed.GetDimension(), sitk.sitkEuler) #this is our second transform, it will return the dimensions of the transform, using the euler transform
tx2.SetParameters( [0.0, 0.0, -np.pi/180.0*10.0, 0.0, 0.0, 0.0] ) # we have 6 degrees of freedom, 3 in the xyz, and 3 rotational
tx2.SetFixedParameters(center)

# Adding our transforms to the transform methof
composite = sitk.Transform(fixed.GetDimension(), sitk.sitkComposite)
composite.AddTransform(tx)
composite.AddTransform(tx2)

In [ ]:
# using the resample function to align the 'moving' image with the fixed using the manual transform defined above
transformed = resample(fixed, moving, composite)

# Write out the image to visual identify quality of transformation in ITK Snap
sitk.WriteImage(transformed, os.path.join(data_directory, 'nPre5.2C0016343_Init.nii'))

## Conduct Registration on Moving and Fixed Images

In [ ]:
R = sitk.ImageRegistrationMethod() # Set up registration method
R.SetMetricAsCorrelation() # define the registration metric as correlation

R.SetOptimizerAsGradientDescent( # define the optimizer as gradient descent and specify parameters of the optimization
    learningRate=2.0,
    numberOfIterations=100,
    convergenceMinimumValue=1e-6,
    convergenceWindowSize=10,
    estimateLearningRate=sitk.ImageRegistrationMethod.EachIteration,
    maximumStepSizeInPhysicalUnits=0.0
)
R.SetInitialTransform(composite) # set the initial transformation values as the manually predefined ones above 
R.SetInterpolator(sitk.sitkLinear) # setting the interpolator as linear 

# setting up smoothing and shirnkage factors for the registrations
R.SetShrinkFactorsPerLevel([4, 2, 1])
R.SetSmoothingSigmasPerLevel([0.1, 0.05, 0.0])
R.SmoothingSigmasAreSpecifiedInPhysicalUnitsOn()

# using the above defined function to output iterative optimization values
R.AddCommand( sitk.sitkIterationEvent, lambda: command_iteration(R) )

# Calculate final transform
print('Starting registration...')
final_trfm = R.Execute(fixed, moving)

In [ ]:
#using the resample with the final transform, moving the image through the tranform
transformed = resample(fixed, moving, final_trfm)

# Writing out image to be used in ITK Snap
sitk.WriteImage(transformed, os.path.join(data_directory, 'nPre5.2C0016343_final.nii'))

## Segmentation and Overlay

In [ ]:
## code to run to overlay and segment
post52seg = segment(fixed, threshold) #segment fixed image
pre52seg = segment(transformed, threshold) # segment transformed image

overlayImage1 = overlay(post52seg, pre52seg) # Overlay images - using specified commented method in defined function

#plot to visualize
plotAll(fixed, plot_slice)
plotAll(moving, plot_slice)
plotAll(post52seg, plot_slice)
plotAll(pre52seg, plot_slice)
plotAll(overlayImage1, plot_slice)

## Calculation of Overlay Percentage

In [ ]:
#Number of pixels (bone) in 5.2 preloading sample
preimage = nib.load(os.path.join(data_directory, 'nPre5.2C0016343_mask.nii'))
preimg_data_arr = np.asarray(preimage.get_data())
print(np.count_nonzero(preimg_data_arr))

#Number of pixels (bone) in 5.2 postloading sample
postimage = nib.load(os.path.join(data_directory, 'nPost5.2C0016420_mask.nii'))
postimg_data_arr = np.asarray(postimage.get_data())
print(np.count_nonzero(postimg_data_arr))

#Number of pixels (bone) in 5.2 overlay, only the the overlapping/alligned areas
OL52 = nib.load(os.path.join(data_directory, '5.2_overlay.nii'))
OLA52 = np.asarray(OL52.get_data())
print(np.count_nonzero(OLA52))

# % 5.2 overlay over pre-loading 
prepercent52 = (np.count_nonzero(OLA52))*100/(np.count_nonzero(preimg_data_arr))
print(prepercent52)

# % 5.2 overlay over post-loading 
postpercent52 = (np.count_nonzero(OLA52))*100/(np.count_nonzero(postimg_data_arr))
print(postpercent52)